In [5]:
import pandas as pd
import numpy as np

# ---------------------------------------------------------------------
# CONFIG
# ---------------------------------------------------------------------

DATA_DIR = "metrics"  # adjust if needed

COMBINED_PATH = f"../{DATA_DIR}/combined_metrics.csv"
BEST_BY_SKU_PATH = f"../{DATA_DIR}/best_by_sku.csv"

SCORE_COL = "score"
BIAS_COL = "bias"
MODEL_COL = "model"
BEST_MODEL_COL = "best_model"
SKU_COL = "id"

# optional if present
STRUCTURE_COLS = ["cv_bin", "adi_bin", "regime"]


In [6]:
# ---------------------------------------------------------------------
# LOAD DATA
# ---------------------------------------------------------------------

combined = pd.read_csv(COMBINED_PATH)
best_by_sku = pd.read_csv(BEST_BY_SKU_PATH)

# sanity
assert SCORE_COL in combined.columns
assert MODEL_COL in combined.columns

print("\nLoaded data:")
print(f"  combined_metrics rows: {len(combined):,}")
print(f"  best_by_sku rows: {len(best_by_sku):,}")
print(best_by_sku.head())
print(best_by_sku.columns)
# 


Loaded data:
  combined_metrics rows: 5,346
  best_by_sku rows: 297
   Unnamed: 0                                            id  \
0           0    CID0_SID0_PID104_MGID6_CAT120-CAT250-CAT32   
1           1     CID0_SID0_PID117_MGID6_CAT14-CAT228-CAT31   
2           2   CID0_SID0_PID118_MGID6_CAT14-CAT228-CAT3180   
3           3  CID0_SID0_PID122_MGID6_CAT120-CAT268-CAT3127   
4           4  CID0_SID0_PID127_MGID6_CAT120-CAT258-CAT3172   

                              best_model         mae       bias       score  \
0                                   Holt   74.144648  14.968214   89.112862   
1                                   Holt  106.649314  -8.760259  115.409573   
2  SeasonalExponentialSmoothingOptimized   37.389926  -2.731973   40.121899   
3                            HoltWinters   81.265023 -56.315274  137.580298   
4                               chronos2   21.079993 -15.380333   36.460327   

        ADI       CV2   regime  
0  1.000000  0.197176  Low-Low  
1  1.000000

In [7]:
# ---------------------------------------------------------------------
# 1. PORTFOLIO-LEVEL PERFORMANCE
# ---------------------------------------------------------------------

portfolio_summary = (
    combined
    .groupby(MODEL_COL)
    .agg(
        mean_score=(SCORE_COL, "mean"),
        p90_score=(SCORE_COL, lambda x: np.percentile(x, 90)),
        mean_abs_bias=(BIAS_COL, lambda x: np.mean(np.abs(x))),
        std_score=(SCORE_COL, "std")
    )
    .sort_values("mean_score")
)

print("\n=== Portfolio-Level Performance ===")
print(portfolio_summary.round(2))


=== Portfolio-Level Performance ===
                                       mean_score  p90_score  mean_abs_bias  \
model                                                                         
DynamicOptimizedTheta                       66.89     153.42          22.06   
SimpleExponentialSmoothingOptimized         67.31     149.12          22.38   
chronos2                                    67.65     142.10          23.89   
Theta                                       67.68     157.11          22.54   
DynamicTheta                                67.69     159.28          22.53   
CrostonOptimized                            67.88     151.53          22.55   
CrostonClassic                              68.36     154.96          23.08   
OptimizedTheta                              68.59     149.67          23.15   
CrostonSBA                                  70.99     158.04          25.13   
SeasonalExponentialSmoothingOptimized       71.40     160.36          23.95   
HoltWinters    

In [8]:
# ---------------------------------------------------------------------
# 2. WIN-RATE (BEST-MODEL SHARE)
# ---------------------------------------------------------------------

win_rate = (
    best_by_sku
    .groupby(BEST_MODEL_COL)
    .size()
    .rename("sku_wins")
    .to_frame()
)

win_rate["win_share"] = win_rate["sku_wins"] / win_rate["sku_wins"].sum()

print("\n=== SKU Win Share ===")
print(win_rate.sort_values("win_share", ascending=False).round(3))



=== SKU Win Share ===
                                       sku_wins  win_share
best_model                                                
WindowAverage                                41      0.138
HistoricAverage                              31      0.104
HoltWinters                                  30      0.101
Holt                                         24      0.081
chronos2                                     23      0.077
SeasonalExponentialSmoothingOptimized        20      0.067
Naive                                        19      0.064
CrostonSBA                                   17      0.057
lightgbm                                     16      0.054
CrostonClassic                               15      0.051
RandomWalkWithDrift                          14      0.047
OptimizedTheta                               12      0.040
SimpleExponentialSmoothingOptimized          11      0.037
Theta                                         8      0.027
SeasonalNaive                    

In [9]:
# ---------------------------------------------------------------------
# 3. WIN-RATE vs PORTFOLIO LOSS RELATIONSHIP
# ---------------------------------------------------------------------

win_vs_loss = (
    portfolio_summary
    .merge(win_rate, left_index=True, right_index=True, how="left")
    .fillna(0)
)

correlation = win_vs_loss["win_share"].corr(win_vs_loss["mean_score"])

print("\n=== Win-Rate vs Mean Score ===")
print(win_vs_loss[["mean_score", "win_share"]].round(3))
print(f"\nCorrelation (win_share vs mean_score): {correlation:.3f}")


=== Win-Rate vs Mean Score ===
                                       mean_score  win_share
model                                                       
DynamicOptimizedTheta                      66.889      0.017
SimpleExponentialSmoothingOptimized        67.313      0.037
chronos2                                   67.655      0.077
Theta                                      67.681      0.027
DynamicTheta                               67.686      0.010
CrostonOptimized                           67.883      0.010
CrostonClassic                             68.357      0.051
OptimizedTheta                             68.591      0.040
CrostonSBA                                 70.989      0.057
SeasonalExponentialSmoothingOptimized      71.397      0.067
HoltWinters                                71.839      0.101
WindowAverage                              73.384      0.138
Holt                                       74.045      0.081
SeasonalNaive                              76.740    

In [10]:
# ---------------------------------------------------------------------
# 4. STRUCTURE-CONDITIONAL PERFORMANCE
# ---------------------------------------------------------------------

structure_cols = [c for c in STRUCTURE_COLS if c in combined.columns]

if structure_cols:
    print("\n=== Structure-Conditional Analysis ===")
    for col in structure_cols:
        print(f"\n-- Performance by {col} --")
        table = (
            combined
            .groupby([col, MODEL_COL])[SCORE_COL]
            .mean()
            .unstack()
        )
        print(table.round(2))
else:
    print("\nNo explicit structure columns found (cv_bin / regime).")



No explicit structure columns found (cv_bin / regime).


In [11]:
# ---------------------------------------------------------------------
# 5. TAIL-RISK DOMINANCE
# ---------------------------------------------------------------------

tail_ratio = (
    portfolio_summary["p90_score"] /
    portfolio_summary["mean_score"]
).rename("p90_to_mean_ratio")

print("\n=== Tail-Risk Ratio (P90 / Mean) ===")
print(tail_ratio.sort_values())


=== Tail-Risk Ratio (P90 / Mean) ===
model
HistoricAverage                          2.088522
RandomWalkWithDrift                      2.090376
chronos2                                 2.100314
WindowAverage                            2.130658
OptimizedTheta                           2.182137
Naive                                    2.212580
SimpleExponentialSmoothingOptimized      2.215386
HoltWinters                              2.224150
CrostonSBA                               2.226293
CrostonOptimized                         2.232262
Holt                                     2.237843
SeasonalExponentialSmoothingOptimized    2.246022
CrostonClassic                           2.266940
DynamicOptimizedTheta                    2.293602
lightgbm                                 2.296927
SeasonalNaive                            2.308343
Theta                                    2.321357
DynamicTheta                             2.353238
Name: p90_to_mean_ratio, dtype: float64


In [12]:
# ---------------------------------------------------------------------
# 6. CHRONOS2 DESCRIPTIVE POSITIONING
# ---------------------------------------------------------------------

if "chronos2" in portfolio_summary.index:
    chronos_row = portfolio_summary.loc["chronos2"]
    chronos_win = win_rate.loc["chronos2"] if "chronos2" in win_rate.index else None

    print("\n=== Chronos2 Positioning ===")
    print(chronos_row.round(2))
    if chronos_win is not None:
        print("\nChronos2 win share:")
        print(chronos_win.round(3))
else:
    print("\nChronos2 not found in model list.")


=== Chronos2 Positioning ===
mean_score        67.65
p90_score        142.10
mean_abs_bias     23.89
std_score         68.78
Name: chronos2, dtype: float64

Chronos2 win share:
sku_wins     23.000
win_share     0.077
Name: chronos2, dtype: float64


In [14]:
# ---------------------------------------------------------------------
# 7. OPTIONAL: EXPORT TABLES FOR SLIDES
# ---------------------------------------------------------------------

portfolio_summary.round(3).to_csv(f"../{DATA_DIR}/analysis_portfolio_summary.csv")
win_vs_loss.round(3).to_csv(f"../{DATA_DIR}/analysis_win_vs_loss.csv")

print("\nAnalysis tables exported:")
print("  analysis_portfolio_summary.csv")
print("  analysis_win_vs_loss.csv")



Analysis tables exported:
  analysis_portfolio_summary.csv
  analysis_win_vs_loss.csv
